In [1]:
# 导入相关的库
import os
import time
import emoji
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from selenium.webdriver.chrome.options import Options

In [2]:
def login():
    gate = "https://www.icourse163.org/"

    person = {
        # "account" : "13539651927"     ,
        # "password": "qazmlp123456@...",
        "account" : "13501408005"     ,
        "password": "qazwsx123520",
    }

    driver = webdriver.Chrome()
    driver.maximize_window()

    # 避开 chrome 检测
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument",
                            {"source": """Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"""})

    driver.get(gate)

    # 登录
    driver.find_element(by=By.CSS_SELECTOR,value="#j-topnav > div > a").click()
    # 其他登录方式
    driver.find_element(by=By.CSS_SELECTOR,value="div.ux-login-set-scan-code_ft > span").click()
    # 手机号登录
    driver.find_element(by=By.CSS_SELECTOR,value="div.ux-tabs-underline > ul > li:nth-child(2)").click()
    driver.switch_to.frame(driver.find_element(by=By.CSS_SELECTOR,value="div.ux-login-urs-phone-wrap.f-pr > div > iframe"))

    # 输入账号
    driver.find_element(by=By.XPATH,value="/html/body/div[2]/div[2]/div[2]/form/div/div[2]/div[2]/input").clear()
    driver.find_element(by=By.XPATH,value="/html/body/div[2]/div[2]/div[2]/form/div/div[2]/div[2]/input").send_keys(person["account"])
    # 输入密码
    driver.find_element(by=By.XPATH,value="/html/body/div[2]/div[2]/div[2]/form/div/div[4]/div[2]/input[2]").clear()
    driver.find_element(by=By.XPATH,value="/html/body/div[2]/div[2]/div[2]/form/div/div[4]/div[2]/input[2]").send_keys(person["password"])

    time.sleep(0.5)
    # 登录
    driver.find_element(by=By.XPATH,value="//div[@class='m-container']/div[@class='u-tab f-cb']//following::div[@class='f-cb loginbox']").click()

    time.sleep(1)
    return driver

In [3]:
# 进入两百个选课
df_200 = pd.read_excel('200个选课_V4.xlsx')

# 合并一个课程名称（开课单位-开课老师）
course_name = df_200['课程名称'].tolist()
course_unit = df_200['开课单位'].tolist()
course_teachers = df_200['开课老师'].tolist()

new_names = [f"{it1}({it2}-{it3})" for it1,it2,it3 in zip(course_name,course_unit,course_teachers)]
course200_urls = df_200['课程url'].tolist()

In [79]:
# 记录不能爬取的课程和url
not_able_to_scrapy_course = []
not_able_to_scrapy_url = []

In [91]:
# 首先打开每一个课程的网页
driver = login()
# # 记录不能爬取的课程和url
# not_able_to_scrapy_course = []
# not_able_to_scrapy_url = []
for url,name in zip(course200_urls[20:],new_names[20:]):
    # 首先先输出课程的名字
    print(name)
    driver.execute_script("window.open('{}');".format(url))
    # 切换到新打开的课程页面
    driver.switch_to.window(driver.window_handles[-1])
    # 已参加，继续学习
    try:
        driver.find_element(by=By.CSS_SELECTOR,value="div.course-enroll-info_course-enroll_buttons > div > span").click()
        # 关掉弹窗
        driver.find_element(by=By.CSS_SELECTOR,value="div.ux-modal_hd > a").click()
        time.sleep(0.5)
        # 点击讨论区
        driver.find_element(by=By.XPATH,value='/html/body/div[4]/div[2]/div[4]/div[1]/div[1]/ul/li[@data-name="讨论区"]').click()
        # 查看一下是否有页数
        time.sleep(0.5) # 这里很关键
        # df1的所有列
        all_forum_lst = []
        all_forum_boolean_lst = []
        all_forum_questions_lst = []
        forum_skim_num = []
        forum_comment_num = []
        forum_goods_num = []

        # 创建一个列表用来存储sheet dataframe
        df_lst = []
        #try:
        bar = driver.find_elements(by=By.XPATH,value='/html/body/div[4]/div[2]/div[4]/div[2]/div/div[1]/div/div[7]/div/div[2]/div/div[1]/div[2]/div/a') # 最后一个button必定是下一页
        bar_lst = [item.text for item in bar]
        new_page_lst = [item for item in bar_lst if item != '']
        pagenum = new_page_lst[-2]

        for i in range(int(pagenum)):
            current_url = driver.current_url
            # 大框架：讨论区主题翻页
            # 爬取主题的标题，问题（这个得点进去才能实现）
            forum_items = driver.find_elements(by=By.XPATH,value = '/html/body/div[4]/div[2]/div[4]/div[2]/div/div[1]/div/div[7]/div/div[2]/div/div[1]/div[1]/li/div')
            # 获取所有的主题的标题
            forum_items_titles = [damn.text for damn in forum_items]
            forum_items_button = driver.find_elements(by=By.XPATH,value = '/html/body/div[4]/div[2]/div[4]/div[2]/div/div[1]/div/div[7]/div/div[2]/div/div[1]/div[1]/li/div/a')
            #print(forum_items_titles)
            one_page_forum_lst = [forum1.split('/n')[-1] for forum1 in forum_items_titles]
            one_page_boolean_lst = [len(forum1.split('\n'))//2 for forum1 in forum_items_titles]
            forum_skim = driver.find_elements(by=By.XPATH,value='/html/body/div[4]/div[2]/div[4]/div[2]/div/div[1]/div/div[7]/div/div[2]/div/div[1]/div[1]/li/p[1]')
            forum_comment = driver.find_elements(by=By.XPATH,value='/html/body/div[4]/div[2]/div[4]/div[2]/div/div[1]/div/div[7]/div/div[2]/div/div[1]/div[1]/li/p[2]')
            forum_goods = driver.find_elements(by=By.XPATH,value='/html/body/div[4]/div[2]/div[4]/div[2]/div/div[1]/div/div[7]/div/div[2]/div/div[1]/div[1]/li/p[3]')
            one_page_skim = [skim.text.split('：')[1] for skim in forum_skim]
            one_page_comment = [comment.text.split('：')[1] for comment in forum_comment]
            one_page_goods = [goods.text.split('：')[1] for goods in forum_goods]
            forum_comment_num += one_page_comment
            forum_goods_num += one_page_goods
            forum_skim_num += one_page_skim

            all_forum_lst += one_page_forum_lst
            all_forum_boolean_lst += one_page_boolean_lst

            # 点击对应的主题 然后进入网页然后再回退
            for index,comment_num in zip([i for i in range(len(forum_items_button))],one_page_comment):
                re_forum_items_button = driver.find_elements(by=By.XPATH,value = '/html/body/div[4]/div[2]/div[4]/div[2]/div/div[1]/div/div[7]/div/div[2]/div/div[1]/div[1]/li/div/a')
                re_forum_items_button[index].click()
                time.sleep(1)
                # 首先获取问题
                try:
                    question_ = driver.find_element(by = By.XPATH,value = '/html/body/div[4]/div[2]/div[4]/div[2]/div/div[1]/div/div[2]/div/div[1]/div/div[2]/p').text
                except:
                    question_ = driver.find_element(by=By.XPATH,value='/html/body/div[4]/div[2]/div[4]/div[2]/div/div[1]/div/div[2]/div/div[1]/div/div[2]').text
                #print(question_)
                all_forum_questions_lst.append(question_)
                # 对于每一个主题，我们需要创建一个dataframe，作为sheet的内容
                tp_df = pd.DataFrame()
                tp_comments = []
                teachers_bollean = []
                thumbs_num_lst = []
                # 如果有回复 我们才进行爬取回复,还需要进行区分回复数与15之间的关系，大于15实际上会有进度条的，需要进行翻页操作
                #print(comment_num)
                if int(comment_num) != 0 and int(comment_num) <= 15:
                    # 不用进行翻页操作，只需爬取当前页面的所有信息并装入即可
                    comments_blocks = driver.find_elements(by=By.XPATH,value = '/html/body/div[4]/div[2]/div[4]/div[2]/div/div[1]/div/div[2]/div/div[4]/div/div[1]/div[1]/div/div[1]/div[2]')
                    comments_neirong = [com.text for com in comments_blocks]
                    tp_comments += comments_neirong
                    teachers_or_not = driver.find_elements(by=By.XPATH,value='/html/body/div[4]/div[2]/div[4]/div[2]/div/div[1]/div/div[2]/div/div[4]/div/div[1]/div[1]/div/div/div[3]/div[1]/span/span[2]')
                    teachers_or_not_mm = [bo.text for bo in teachers_or_not]
                    teachers_bollean += [1 if item[-2:]=='老师' else 0 for item in teachers_or_not_mm]
                    thumbs_num_button = driver.find_elements(by=By.XPATH,value='/html/body/div[4]/div[2]/div[4]/div[2]/div/div[1]/div/div[2]/div/div[4]/div/div[1]/div[1]/div/div/div[3]/div[7]/div/p')
                    thumbs_num_ = [item.text for item in thumbs_num_button]
                    thumbs_num_lst += thumbs_num_
                    # print(thumbs_num_)
                    tp_df['回复'] = tp_comments
                    tp_df['是否为老师'] = teachers_bollean
                    tp_df['点赞数'] = thumbs_num_lst
                    df_lst.append(tp_df)

                elif int(comment_num) == 0:
                    # 不需要爬信息，直接装入df_lst
                    tp_df['回复'] = tp_comments
                    tp_df['是否为老师'] = teachers_bollean
                    tp_df['点赞数'] = thumbs_num_lst
                    df_lst.append(tp_df)
                else:
                    # 此处表示的是有翻页操作
                    bar_2 = driver.find_elements(by=By.XPATH,value='/html/body/div[4]/div[2]/div[4]/div[2]/div/div[1]/div/div[2]/div/div[4]/div/div[1]/div[2]/div/a')
                    bar2 = [hahaha.text for hahaha in bar_2]
                    clean_bar2 = [item for item in bar2 if item != '']
                    tp_pagenum = clean_bar2[-2]

                    # 下面开始翻页的操作
                    for j in range(int(tp_pagenum)):
                        bar_hahaha = driver.find_elements(by=By.XPATH,value='/html/body/div[4]/div[2]/div[4]/div[2]/div/div[1]/div/div[2]/div/div[4]/div/div[1]/div[2]/div/a')

                        # 首先先获取信息
                        # 不用进行翻页操作，只需爬取当前页面的所有信息并装入即可
                        comments_blocks = driver.find_elements(by=By.XPATH,value = '/html/body/div[4]/div[2]/div[4]/div[2]/div/div[1]/div/div[2]/div/div[4]/div/div[1]/div[1]/div/div[1]/div[2]')
                        comments_neirong = [com.text for com in comments_blocks]
                        tp_comments += comments_neirong
                        teachers_or_not = driver.find_elements(by=By.XPATH,value='/html/body/div[4]/div[2]/div[4]/div[2]/div/div[1]/div/div[2]/div/div[4]/div/div[1]/div[1]/div/div/div[3]/div[1]/span/span[2]')
                        teachers_or_not_mm = [bo.text for bo in teachers_or_not]
                        damn = [1 if item[-2:]=='老师' else 0 for item in teachers_or_not_mm]

                        teachers_bollean += [1 if item[-2:]=='老师' else 0 for item in teachers_or_not_mm]
                        thumbs_num_button = driver.find_elements(by=By.XPATH,value='/html/body/div[4]/div[2]/div[4]/div[2]/div/div[1]/div/div[2]/div/div[4]/div/div[1]/div[1]/div/div/div[3]/div[7]/div/p')
                        thumbs_num_ = [item.text for item in thumbs_num_button]
                        thumbs_num_lst += thumbs_num_
                        # print(thumbs_num_)
                        if j != (int(tp_pagenum)-1):
                            bar_hahaha[-1].click()
                            time.sleep(0.5)

                    tp_df['回复'] = tp_comments
                    tp_df['是否为老师'] = teachers_bollean
                    tp_df['点赞数'] = thumbs_num_lst
                    df_lst.append(tp_df)


                # 进行回退的操作
                driver.execute_script("window.open('{}');".format(current_url))
                driver.close()
                handles = driver.window_handles
                driver.switch_to.window(handles[-1])
                time.sleep(1)

            # 点击下一页
            if i != int(pagenum)-1:
                bar = driver.find_elements(by=By.XPATH,value='/html/body/div[4]/div[2]/div[4]/div[2]/div/div[1]/div/div[7]/div/div[2]/div/div[1]/div[2]/div/a')
                bar[-1].click()
                time.sleep(0.5)

        #创建df1的东西
        df1 = pd.DataFrame()
        hhh = [item.split('\n')[1] if len(item.split('\n'))==2 else item for item in all_forum_lst]
        df1['讨论区主题'] = hhh
        df1['老师是否参与'] = all_forum_boolean_lst
        df1['讨论主题问题'] = all_forum_questions_lst
        df1['浏览数'] = forum_skim_num
        df1['评价数'] = forum_comment_num
        df1['点赞数'] = forum_goods_num

        # 下面创建all_df(通过name)
        new_wb = pd.ExcelWriter(f'./讨论区评论/{name}.xlsx')
        df1.to_excel(new_wb,sheet_name='主题')
        hhh2 = [(i+1) for i in range(len(hhh))]
        for tp_df,na in zip(df_lst,hhh2):
            na_ = f"第{na}个主题"
            tp_df.to_excel(new_wb,sheet_name=na_)

        new_wb.save()
        new_wb.close()

    except:
        not_able_to_scrapy_course.append(name)
        not_able_to_scrapy_url.append(url)

    driver.close()
    handles = driver.window_handles
    driver.switch_to.window(handles[-1])
    #break

核医学(华中科技大学-兰晓莉)
中医气功学导论(北京中医药大学-魏玉龙)
德国历史十二讲(北京师范大学-孙立新)
城市与文化遗产(武汉大学-夏增民)
《资治通鉴》导读(复旦大学-姜鹏)
诸子的智慧(南开大学-张荣明)
孙子兵法鉴赏(福州大学-陈润华)
走近中华优秀传统文化(南京大学-张亮)
中华优秀传统文化(复旦大学-唐明燕)
老子的人生智慧(东北大学-张雷)
学术交流英语(东南大学-陈美华)
交替传译(上海外国语大学-王天星)
学术英语写作(北京工业大学-万伟珊)
俄罗斯文化之旅(厦门大学-徐琪)
综合日语实践——基础日语1(上海外国语大学-周异夫)
德语入门：基础语音和会话(南京大学-常晅)
美国诺奖作家经典赏析(首都经济贸易大学-张龙海)
英语演讲(上海外国语大学-王睿)
德语国家社会与文化(华东师范大学-杨建培)
BIM技术与应用(上海大学-汪德江)
化工安全（上）(华东理工大学-李涛)
数字信号处理(北京航空航天大学-王俊)
自动控制元件(沈阳工业大学-魏民)
制造业信息化导论(浙江大学-纪杨建)
电路基础(西北工业大学-段哲民)
食品化学与营养(福州大学-倪莉)
理论力学(西安交通大学-张娟)
数字信号处理(西安电子科技大学-田春娜)
物理性污染控制(福建农林大学-范立维)
模拟电子技术(沈阳工业大学-于海雁)
风景园林景观规划设计基本原理(同济大学-刘滨谊)
模拟电子技术基础(西安交通大学-赵进全)
画法几何及土木工程制图（上）(大连理工大学-王子茹)
电机与拖动(北京信息科技大学-厉虹)
无人驾驶车辆(北京理工大学-熊光明)
传感器与测试技术(国防科技大学-叶湘滨)
荷载与结构设计原理(武汉理工大学-张松)
隧道工程(中南大学-王薇)
汽车发动机构造(天津大学-高文志)
测量学(电子科技大学-何彬彬)
无人机设计导论(南京航空航天大学-昂海松)
化学实验室安全科学(西安交通大学-郭栋才)
信号与系统(北京邮电大学-尹霄丽)
钢结构基本原理(同济大学-赵宪忠)
现代工程制图（下）(大连理工大学-王丹虹)
MATLAB语言基础(北京邮电大学-俎云霄)
动力工程(华北电力大学（保定）-吴正人)
DIY智慧小屋—带你玩转物联网(北京大学-史治国)
电磁场与电磁波(北京交通大学-邵小桃)
模拟电子技术(北京交通大学-刘颖)
工业生态学(东北大学-杜涛)
机器学

In [78]:
len(not_able_to_scrapy_course)

20

In [88]:
#not_able_to_scrapy_course = not_able_to_scrapy_course[:5] + not_able_to_scrapy_course[6:]
not_able_to_scrapy_url = not_able_to_scrapy_url[:5] + not_able_to_scrapy_url[6:]

In [92]:
df_not_able_to_scarpy = pd.DataFrame()
df_not_able_to_scarpy['课程名称'] = not_able_to_scrapy_course
df_not_able_to_scarpy['课程url'] = not_able_to_scrapy_url

df_not_able_to_scarpy

,课程名称,课程url
0,动物学(华中农业大学-姬伟),https://www.icourse163.org/course/HZAU-1002478006
1,茶叶感官审评(安徽农业大学-戴前颖),https://www.icourse163.org/course/AHAU-1206699836
2,禽生产学(西北农林科技大学-王哲鹏),https://www.icourse163.org/course/NWSUAF-12066...
3,卫生应急管理(哈尔滨医科大学-吴群红),https://www.icourse163.org/course/HEBYKDX-1454...
4,妇产科护理学(湖南师范大学-陈丹),https://www.icourse163.org/course/HUNNU-120751...
...,...,...
135,程序设计入门——C语言(浙江大学-翁恺),https://www.icourse163.org/course/ZJU-199001
136,数据结构(浙江大学-陈越),https://www.icourse163.org/course/ZJU-93001
137,面向对象程序设计——Java语言(浙江大学-翁恺),https://www.icourse163.org/course/ZJU-1001542001
138,C语言程序设计进阶(浙江大学-翁恺),https://www.icourse163.org/course/ZJU-200001


In [ ]:
df_not_able_to_scarpy.to_excel('./讨论区评论/暂时不能爬取的课程.xlsx')